In [63]:
import requests
from bs4 import BeautifulSoup as bs
import lxml

In [64]:
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import time
import requests
from selenium.webdriver.common.keys import Keys

In [65]:
import sqlite3
import pandas as pd

In [66]:
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

In [67]:
path = 'C:\\Users\\User\Desktop\\새 폴더\\chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.
#       내 컴퓨터는 \를 두개씩쳐야됨
driver = webdriver.Chrome(path)

# 기다려줍시다
driver.implicitly_wait(3) #혹은 time.sleep(3)

In [68]:
def get_daily_boxoffice(btn):

    options = webdriver.ChromeOptions()

    options.add_argument('headless')

    options.add_argument('window-size=1920x1080')  #잇으나없으나의미없는코드래요 뭔말인지모르겟어요저는으아아아아아

    driver = webdriver.Chrome(path, options=options)

    driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")

    driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

    driver.find_element_by_id(btn).click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

    driver.implicitly_wait(4)

    driver.find_element_by_id(btn).click()

    driver.implicitly_wait(4)

    driver.find_element_by_id(btn).click()

    driver.implicitly_wait(4)

    driver.find_element_by_id(btn).click()

    daily_boxoffice_html = driver.page_source

    driver.quit()

    parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

    daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_0"})

    movie_list = daily_boxoffice.find_all("tr")

    daily_movies = {'영화명' : [],
    '순위 변동' : [],
    '개봉일' : [],
    '매출액' : [],
    '매출액점유율' : [],
    '관객수' : [],
    '누적관객수' : [],
    '스크린수' : [],
    '상영횟수' : []}

    for movie in movie_list:
        title_rank = movie.find("td").find_next_sibling("td").text
        daily_movies['영화명'].append(title_rank[:-5].strip())
        daily_movies['순위 변동'].append(title_rank[-4:].strip())
        daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())

    daily_boxoffice = pd.DataFrame(daily_movies)

    daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))
    
    return daily_boxoffice

In [69]:
btnList = ['btn_0', 'btn_112', 'btn_240', 'btn_357', 'btn_467', 'btn_572', 'btn_669' ]
boxofficeList = []
for btn in btnList:
    boxofficeList.append(get_daily_boxoffice(btn))

In [38]:
boxofficeList[1]

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,장사리 : 잊혀진 영웅들,0,2019-09-25,"1,730,179,400",33.7%,"201,971","497,843","1,064","4,814"
1,양자물리학,1,2019-09-25,"964,521,750",18.8%,"110,099","276,855",772,"2,728"
2,나쁜 녀석들: 더 무비,-1,2019-09-11,"957,282,740",18.7%,"109,811","4,353,000","1,000","4,024"
3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"467,303,790",9.1%,"52,383","154,414",678,"1,931"
4,애드 아스트라,0,2019-09-19,"176,380,740",3.4%,"19,956","470,595",493,919
5,퍼펙트맨,1,2019-10-02,"147,407,700",2.9%,"17,495","74,204",481,723
6,타짜: 원 아이드 잭,-1,2019-09-11,"107,076,860",2.1%,"11,728","2,202,591",405,907
7,더 룸,-1,2019-09-25,"89,743,700",1.7%,"10,108","36,458",163,496
8,예스터데이,-1,2019-09-18,"72,460,220",1.4%,"8,237","285,543",317,470
9,극장판 헬로카봇 : 달나라를 구해줘!,7,2019-09-04,"51,966,600",1.0%,"6,644","569,399",172,241


In [71]:
con = sqlite3.connect("daily_boxoffice.db") #데이터베이스하나만드는코드


dates = ['28','27','26', '25', '24', '23', '22']

for date in dates:
    
    con.execute("CREATE TABLE IF NOT EXISTS boxoffice_{0}(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)".format(date))

    daily_boxoffice.to_sql("boxoffice_{0}".format(date), con, chunksize=1000, if_exists='replace')

con.commit()

con.close()

TypeError: 'NoneType' object is not callable

In [70]:
con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_22")

data = cursor.fetchall()

pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,0,2019-09-25,"1,730,179,400",33.7%,"201,971","497,843","1,064","4,814"
1,1,양자물리학,1,2019-09-25,"964,521,750",18.8%,"110,099","276,855",772,"2,728"
2,2,나쁜 녀석들: 더 무비,-1,2019-09-11,"957,282,740",18.7%,"109,811","4,353,000","1,000","4,024"
3,3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"467,303,790",9.1%,"52,383","154,414",678,"1,931"
4,4,애드 아스트라,0,2019-09-19,"176,380,740",3.4%,"19,956","470,595",493,919
5,5,퍼펙트맨,1,2019-10-02,"147,407,700",2.9%,"17,495","74,204",481,723
6,6,타짜: 원 아이드 잭,-1,2019-09-11,"107,076,860",2.1%,"11,728","2,202,591",405,907
7,7,더 룸,-1,2019-09-25,"89,743,700",1.7%,"10,108","36,458",163,496
8,8,예스터데이,-1,2019-09-18,"72,460,220",1.4%,"8,237","285,543",317,470
9,9,극장판 헬로카봇 : 달나라를 구해줘!,7,2019-09-04,"51,966,600",1.0%,"6,644","569,399",172,241
